In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:

# Write any helper functions you need
def phaseshift(angle):
    M=np.zeros((2,2),dtype=complex)
    M[0,0]=np.exp(1j*angle)
    M[1,1]=np.exp(-1j*angle)
    return M
    
def U_op(a):
    M=np.zeros((2,2),dtype=complex)
    M[0,0]=a
    M[1,1]=-a
    M[0,1]=np.sqrt(1-a**2)
    M[1,0]=np.sqrt(1-a**2)
    return M

def coefficients(angles):
    
    """ This function returns the coefficients associated with the polynomial generated by 
    the QSP routine as a function of the phase angles.
    
    Args:
        - angles (np.array(float)): Array of real numbers containing the four phase angles,
        in reverse order of application.
    
    Returns:       
        - (np.array(complex)): A numpy array containing the coefficients of the polynomial
        generated by QSP, where the first element is the coefficient of the cubic term and
        the second element is for the linear term.
    """
    

    # Put your code here
    U=phaseshift(angles[0])
    a=1
    for i in range(3):
        U=np.dot(U_op(a),U)
        U=np.dot(phaseshift(angles[i+1]),U)

    A=U[0,0]

    a=np.exp(1j*2*np.pi/3)
    U=phaseshift(angles[0])
    for i in range(3):
        U=np.dot(U_op(a),U)
        U=np.dot(phaseshift(angles[i+1]),U)
        
    B=U[0,0]

    beta=(B-A)/(np.exp(1j*2*np.pi/3)-1)

    alpha=A-beta

    return alpha,beta

# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    coeffs = coefficients(ins)
    outs = [[np.real(elem).numpy(), np.imag(elem).numpy()] for elem in coeffs]
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol = 1e-4)

In [3]:
# These are the public test cases
test_cases = [
    ('[0.5,0.8,1.0,1.0]', '[[-0.1707976, -2.4084889], [-0.8166822, 2.2507432]]'),
    ('[-0.20409113, -0.91173829, 0.91173829, 0.20409113]', '[[2.5,0],[-1.5,0]]')
]

In [4]:
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.5,0.8,1.0,1.0]'...
Correct!
Running test case 1 with input '[-0.20409113, -0.91173829, 0.91173829, 0.20409113]'...
Correct!
